In [1]:
%matplotlib widget

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyodbc
import os
import time
from matplotlib import colors
from scipy import stats

Connection to sq server...

In [3]:
server = 'mrwhite.intrans.iastate.edu'
database = 'sim_Andalib'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes')

//Fonts for graphs. 

In [4]:
font = {'family' : 'serif',
        'serif' : 'Times New Roman',
        'size'   : 8}
plt.rc('font', **font)

Define number of intersections

Coordinated signal groups

Travel time sections to calculate vehicle arrival

study Id: 2

Run ID: 1-model: 38th street. cycle length: 120; offset zero

In [5]:
### input parameters
number_of_intersection = 8

sim_id = 20 
run_id = 1
iteration = 1

cycle_length = 120

simulation_time = 3600
number_of_cycle = simulation_time//cycle_length
cycle_remainder = simulation_time%cycle_length

signal_timeStep = 1
travel_time_timestep = 1

In [6]:
filePath = "C:\\Users\\ashams\\Box\\research works\\comparison signal offset optimization\\bandwidth maximization\\"
input_parameters = pd.read_excel(filePath + "input_parameters.xlsx", sheet_name = "38th_subset")

input_parameters.loc[:, 'nextIntersection'] = input_parameters.LinkToInt
input_parameters.loc[input_parameters.sg_id == 6, 'nextIntersection'] -= 2


# input_parameters.loc[:, "offset_loc"] = 0
# input_parameters.loc[:, "offset_up"] = 0
input_parameters.LinkFromInt -= 1
input_parameters.LinkToInt -= 1

input_parameters.loc[:, "travel time"] = np.round(input_parameters.loc[:, "travel time"] , 0)
input_parameters.loc[:, "travel time"] = input_parameters.loc[:, "travel time"]%cycle_length

input_parameters

,Intersection Number,intersection_id,sg_id,tt_id,LinkFromInt,LinkToInt,travel_time,saturation flow,number of lane,vpb,distance,speed,tt,split,travel time,d1,tt_next_intersection,d1_next_intersection,nextIntersection
0,9005,1,2,30503,-1,0,5,1850,3,1.541667,0,51.45,0.0,60,0.0,190,12.9,385,1
1,9005,1,6,30504,1,0,5,1850,3,1.541667,662,51.45,12.9,60,0.0,210,0.0,0,-1
2,9006,2,2,30603,0,1,5,1850,3,1.541667,662,51.45,12.9,60,107.0,385,12.8,585,2
3,9006,2,6,30604,2,1,5,1850,3,1.541667,660,51.45,12.8,60,13.0,405,12.9,210,0
4,9007,3,2,30703,1,2,5,1850,3,1.541667,660,51.45,12.8,70,94.0,585,12.8,765,3
5,9007,3,6,30704,3,2,5,1850,3,1.541667,657,51.45,12.8,50,26.0,605,12.8,405,1
6,9008,4,2,30803,2,3,5,1850,3,1.541667,657,51.45,12.8,44,82.0,765,13.1,970,4
7,9008,4,6,30804,4,3,5,1850,3,1.541667,675,51.45,13.1,44,38.0,785,12.8,605,2
8,9009,5,2,30903,3,4,5,1850,3,1.541667,675,51.45,13.1,70,68.0,970,12.8,1170,5
9,9009,5,6,30904,5,4,5,1850,3,1.541667,660,51.45,12.8,70,52.0,990,13.1,785,3


Collect signal status change data from sql server. 

In [7]:
signal_sql_test = "SELECT [SimTime], [SC], [SG], [state] FROM [sim_Andalib].[dbo].[signalchanges] where [StudyID]="+str(sim_id)+"and [RunID]="+str(run_id)+"and [Iteration]="+str(iteration)+ "and [SimTime]<="+str(simulation_time)+ "Order By [SimTime]"
signal = pd.read_sql(signal_sql_test, cnxn)

if(len(signal)==0): print("ERROR")

# 1- green start, 2- yellow start, 0- Red start
signal = signal.loc[signal.state.isin([0, 1])]
signal["eog"] = (signal.SimTime - signal.SimTime*signal.state)*signal_timeStep
signal["bog"] = (signal.SimTime - signal.SimTime*(1-signal.state))*signal_timeStep

Collect travel time data from sql server.

In [8]:
travel_time_sql = "SELECT [Iteration],[Time],[No_],[Veh],[Trav_],[Delay_] FROM [sim_Andalib].[dbo].[traveltimes] where [StudyID]="+str(sim_id)+"and [RunID]="+str(run_id)+"and [Iteration]="+str(iteration)+"and [Time]<="+str(simulation_time)+ "Order By [Time]"
travel_time = pd.read_sql(travel_time_sql, cnxn)
travel_time.columns = ['Iteration','time','tt_id', 'vehNo', 'tt', 'travel_delay']

This section creates distribution of green and also calculates the probability. 

In [9]:
signal["eog"] = signal["eog"].astype(np.int64)
signal["bog"] = signal["bog"].astype(np.int64)

green_time = np.zeros((number_of_intersection,2, (number_of_cycle+1)*cycle_length*signal_timeStep))
green_time_10 = np.zeros((number_of_intersection,2, (number_of_cycle+1)*cycle_length*signal_timeStep))

eog_matrix = np.zeros((number_of_intersection, 2, number_of_cycle+10))
split_matrix = np.zeros((number_of_intersection, 2, number_of_cycle+10))

eog_cycle_num = np.zeros((number_of_intersection, 2), dtype = np.int16)
split_cycle_num = np.zeros((number_of_intersection, 2), dtype = np.int16)


for link_ in range(len(input_parameters)):
    
        intersection_num = input_parameters["Intersection Number"][link_]
        intersection_ = input_parameters["intersection_id"][link_]

        sg_ = input_parameters.sg_id[link_]

        signal_ = signal.loc[(signal.SC==intersection_num) & (signal.SG==sg_)]

        if(sg_==2): bound = 0
        else: bound  = 1
    
        for step in range(0, len(signal_), 2):
                bog = signal_.iloc[step, 5]

                if(step<len(signal_) - 1):
                        eog = signal_.iloc[step+1, 4]

                        if(eog>0.1): 
                                eog_matrix[intersection_ - 1][bound][eog_cycle_num[intersection_ - 1][bound]] = eog%cycle_length
                                eog_cycle_num[intersection_ - 1][bound] += 1

                else: eog = simulation_time

                if(bog>0.1 and eog!=simulation_time):
                        split_matrix[intersection_ - 1][bound][split_cycle_num[intersection_ - 1][bound]] = eog - bog
                        split_cycle_num[intersection_ - 1][bound] += 1    

                green_time[intersection_ -1,bound, bog:eog] = 1
                green_time_10[intersection_ -1,bound, bog+10:eog] = 1
            
green_distribution = green_time.reshape(number_of_intersection,2, (number_of_cycle+1), cycle_length*signal_timeStep)
green_distribution_10 = green_time_10.reshape(number_of_intersection,2, (number_of_cycle+1), cycle_length*signal_timeStep)

cycle_sum = np.ones((cycle_length*signal_timeStep, 1)) * number_of_cycle
cycle_sum[:cycle_remainder*signal_timeStep, 0] += 1

final_probability_of_green = np.sum(green_distribution, axis = 2)/cycle_sum.T
final_Prog_10 = np.sum(green_distribution_10, axis = 2)/cycle_sum.T

green_aog = np.array(final_probability_of_green)
green_AOG_10 = np.array(final_Prog_10)

green_PrOG = np.array(final_probability_of_green)

## Previous definitions
## probability_of_green -> probability to use in AOG and delay calculation
## final_PrOG           -> Probability to use in PrOG calculation


In [10]:
expcted_split = np.round(np.sum(split_matrix, axis = 2)/ split_cycle_num, 0)
expcted_split = np.minimum(expcted_split, cycle_length)

final_green_time = np.zeros((number_of_intersection,2, cycle_length*signal_timeStep))

for i in range(number_of_intersection):
        for j in range(2):
                split_current = split_matrix[i, j, :split_cycle_num[i][j]]
                expcted_split2 = int(np.round(stats.mode(split_current)[0], 0))

                eog = int(stats.mode(eog_matrix[i,j, :eog_cycle_num[i][j]])[0])

                #         bog = (int(expcted_eog[i][j]) - expcted_split2)%cycle_length
                bog = (eog - int(expcted_split[i][j]))%cycle_length

                print(i, " ", j, " ", bog, " ", eog)

                if(eog>bog): 
                        final_green_time[i, j, bog:eog] = 1
                else: 
                        final_green_time[i, j, bog:cycle_length] = 1
                        final_green_time[i, j, 0:eog] = 1

#check if everything is right
assert(np.sum(np.sum(final_green_time, axis = 2) - expcted_split)==0)

0   0   102   57
0   1   102   57
1   0   102   57
1   1   111   57
2   0   109   67
2   1   112   67
3   0   115   41
3   1   107   41
4   0   111   67
4   1   111   67
5   0   101   67
5   1   101   67
6   0   97   67
6   1   97   67
7   0   106   52
7   1   106   52


In [62]:
green_matrix = np.array(green_time[:, :, :simulation_time])

link_ = 0
for i in range(number_of_intersection):
        for j in range(2):
                tt = int(input_parameters["travel time"][link_])

                placeHolder_PrOG  = np.hstack((green_PrOG[i, j, cycle_length - tt: cycle_length], \
                                               green_PrOG[i, j, 0: cycle_length - tt] ))
                placeHolder_green  = np.hstack((final_green_time[i, j, cycle_length - tt: cycle_length], \
                                                final_green_time[i, j, 0: cycle_length - tt]))

                green_PrOG[i, j, :] = placeHolder_PrOG[:]
                final_green_time[i, j, :] = placeHolder_green[:]

                link_ += 1
        
PrOG = np.array(final_probability_of_green)
green = np.array(final_green_time)


In [63]:
### capacity_matrix
capacity = np.zeros((number_of_intersection,2, cycle_length*signal_timeStep))

for link_ in range(len(input_parameters)):
        intersection_ = input_parameters["intersection_id"][link_]
        sg_ = input_parameters.sg_id[link_]
        vpb = input_parameters.vpb[link_]

        if(sg_==2): bound = 0
        else: bound  = 1
        capacity[intersection_ -1,bound, :] = green_aog[intersection_ -1,bound, :] * vpb * number_of_cycle

fin_capacity = np.array(capacity)

This section calculates vehicle arrival distribution. See the excel file "sql_data.xlsx" for better understanding. 

In [64]:
vehicle_arrival = np.zeros((number_of_intersection,2, (number_of_cycle+1)*cycle_length*travel_time_timestep))

for link_ in range(len(input_parameters)):
    
        if(input_parameters.loc[link_, "sg_id"]==2): bound = 0
        else: bound  = 1

        intersection_ = input_parameters.loc[link_, "intersection_id"] -1

        if(input_parameters.loc[link_, "LinkFromInt"]==-1): 
                vehicle_arrival[intersection_,bound, :] = 0
                continue


        travel_time_intersection = input_parameters.loc[link_, "travel_time"]
        travel_time_id = input_parameters.loc[link_, "tt_id"]

        veh_tt = travel_time.loc[travel_time.tt_id == travel_time_id].copy()    
        veh_tt.loc[:, "timestamp"] = np.round((veh_tt.time - veh_tt.tt + travel_time_intersection)*travel_time_timestep, 0)
        veh_tt.timestamp = veh_tt["timestamp"].astype(np.int64)

        unique, counts = np.unique(veh_tt.timestamp, return_counts=True)
        vehicle_arrival[intersection_,bound, unique] = counts

vehicle_arrival_reshape = vehicle_arrival.reshape(number_of_intersection,2, (number_of_cycle+1), cycle_length*travel_time_timestep)
final_vehicle_distribution = np.sum(vehicle_arrival_reshape, axis = 2)

vehicle_distribution = np.array(final_vehicle_distribution)


In [65]:
reset_adjustments(green_aog, vehicle_distribution, capacity)
plot_distributions(green_aog, vehicle_distribution)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
zeros = np.zeros(8, np.int16)
print_output(zeros)
# plot_offset_shift(zeros, direc_tory, "sim_prog_hc_offset", 1800,2700)


System AOG: 14059.57
System POG: 17.56
System Delay: 44.17
System Bandwidth: 41
System Number of Stops: 8771.62


This section will reset the probability of green and vehicle distribution pattern to simulation result. 

This function will shift the probability_of_green, vehicle_distribution for given offsets. 

In [41]:
def make_adjustments(bin_GAdj, bin_VAdj, offsetAdj):
    
        link_ = 0
    
        for intersection in range(number_of_intersection):
                for j in range(2):

                        if(input_parameters.loc[link_, "sg_id"]==2): bound = 0
                        else: bound  = 1

                        next_Intersection = input_parameters.loc[link_, "nextIntersection"]

                        next_intersection_valid = 0
                        if(next_Intersection>=0 and next_Intersection< number_of_intersection):
                                next_intersection_valid = 1

                        shift = int(offsetAdj[intersection])


                        if(next_intersection_valid):
                                placeHolder_VA  = np.hstack(( bin_VAdj[next_Intersection, bound, cycle_length - shift: cycle_length], \
                                                              bin_VAdj[next_Intersection, bound, 0: cycle_length - shift] ))

                                bin_VAdj[next_Intersection, bound, :] = placeHolder_VA[:]


                        placeHolder_GA  = np.hstack(( bin_GAdj[intersection, bound, cycle_length - shift: cycle_length], \
                                                  bin_GAdj[intersection, bound, 0: cycle_length - shift]))

                        bin_GAdj[intersection, bound, :] = placeHolder_GA[:]


                        link_ += 1

In [17]:
def reset_adjustments(pog, veh_dist, capacity_matrix):
        pog[:, :, :] = np.array(final_probability_of_green)
        veh_dist[:, :, :] = np.array(final_vehicle_distribution)
        capacity_matrix[:, :, :] = np.array(fin_capacity)

In [18]:
def green_adjustments(bin_GAdj, offsetAdj):
        for i in range(number_of_intersection):
                offset_value = int(offsetAdj[i])
                for j in range(2):
                        bin_GAdj[i, j, :] = np.hstack(( bin_GAdj[i, j, cycle_length - offset_value: cycle_length], \
                                                    bin_GAdj[i, j, 0: cycle_length - offset_value]))


In [19]:
def reset_green():
        green[:, :, :] = np.array(final_green_time) 
        PrOG[:, :, :] = np.array(green_PrOG)

This fucntion plots probability_of_green and vehicle_distribution

In [20]:
def plot_distributions(probability_of_green, vehicle_distribution):
        signal_seq = np.arange(0,cycle_length, 1/signal_timeStep)
        veh_seq = np.arange(0,cycle_length, 1/travel_time_timestep)

        cols = (number_of_intersection)
        rows = 2

        fig, axes = plt.subplots(nrows = rows, ncols = cols,sharey=True, sharex = True, figsize=(32, 4))

        fig.tight_layout()
    
        for link_ in range(len(input_parameters)):
        
                i = input_parameters["intersection_id"][link_]-1

                if(input_parameters.loc[link_, "sg_id"]==2): j = 1
                else: j  = 0

                ax = axes[j][i]
                ax.set_title("Intersection Number: " + str(i+1) + " SG: " +str(input_parameters.loc[link_, "sg_id"]))
                ax.grid(which = 'both', linestyle = 'dashed')
                ax.grid(b=True, which='minor', alpha=0.2)
                ax.minorticks_on()
                ax.set_axisbelow(True)

                ax.bar(signal_seq,probability_of_green[i][1-j], width = 1/signal_timeStep, color = 'green', alpha = 0.8)

                ax2=ax.twinx()
                ax2.bar(veh_seq,vehicle_distribution[i][1-j], width = 1/travel_time_timestep, color = 'black', alpha = 0.6)
        

In [21]:
reset_adjustments(green_aog, vehicle_distribution, capacity)
plot_distributions(green_aog, vehicle_distribution)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [108]:
offsetAdj = np.array([10, 0, 10, 0, 0, 0, 0, 98], np.int16)
reset_adjustments(green_aog, vehicle_distribution, capacity)
make_adjustments(green_aog, vehicle_distribution, offsetAdj)
plot_distributions(green_aog, vehicle_distribution)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<h1>Objective Functions</h1>

This is from Prof. Day's arrival profile based optimization. Where the objective function is to maximize arrival on green- <br>
total arrival on green = $\sum \limits_{i=1}^{numberOfIntersection}{ \sum \limits_{j=0}^1 {\prod \limits_{k=1}^{cycleLength}{p(G_k) vehDistribution(k)}}}$

In [22]:
def arrival_profile_optimize(bin_GAdj, bin_VAdj): 

        bin_AOG = np.multiply(bin_GAdj, bin_VAdj) 
        sys_AOG = np.sum(bin_AOG)

        return sys_AOG

Arrival Function for $(1, \dots, n)$ intersection- 

$\Gamma_{1, \dots, n}(t, \omega_1, \dots, \omega_i) = \prod \limits_{i=1}^{n}{\Gamma_i(t-\omega_i)}$

Bandwidth definition-

$b(\omega_1, \dots, \omega_n)=\int{\Gamma_{1,\dots, n}(t, \omega_1, \dots, \omega_n)}$


<ul>
   <li> These expressions are for one-way bandwidth.
   <li> For two-way bandwidth the problem is-  max $\zeta = b_1 + b_2 $ 
</ul>
The next cell calculates $\zeta$ for given arrival matrix.

In [23]:
### badnwidth

def maximize_bandwidth(bin_GAdj):
        return np.sum(np.prod(bin_GAdj, axis = 0))

In [24]:
def calculate_delay(capacity_matrix, bin_VAdj):
    ### If (overall) volume exceeds capacity this method will not work
    
        queue= np.zeros(bin_VAdj.shape)

        veh_left = bin_VAdj - capacity_matrix

        ### first iteration    
        for i in range(cycle_length):
                queue[:, :, i] = np.maximum(queue[:, :,i-1] + veh_left[:, :, i], 0)
    
    
    ### second iteration
        for i in range(cycle_length):
                queue[:, :, i] = np.maximum(queue[:, :,i-1] + veh_left[:, :, i], 0)

        delay = np.sum(queue)/3600

        return delay

In [25]:
def number_of_stops(capacity_matrix, bin_VAdj):
    ### If (overall) volume exceeds capacity this method will not work
    
        queue= np.zeros(bin_VAdj.shape)

        veh_left = bin_VAdj - capacity_matrix

        ### first iteration
        for i in range(cycle_length):
                queue[:, :, i] = np.maximum(queue[:, :, i-1] + veh_left[:, :, i], 0)
    
    ### second iteration
        total_stops = 0
    
        for i in range(cycle_length):
                queue[:, :, i]   = np.maximum(queue[:, :, i-1] + veh_left[:, :, i], 0)

                stops = np.maximum(bin_VAdj[:, :, i] - np.maximum(capacity_matrix[:, :, i] - queue[:, :, i-1], 0), 0)

                total_stops += np.sum(stops)

        return total_stops

In [30]:
def performance(offsetAdj, method_to_use):
    ## 1: arrival profile
    ## 2: delay model
    ## 3: Bandwidth maximization (mode_eog - mean_bog)
    ## 4: PrOG-maximize
    ## 5: Number of stops
    ## 6: delay + 20*number of stops
    ## 7: aog (Bog+ 10)
    ## 8: Pareto optimal

        if(method_to_use==1):
        # Reset matrix
                green_aog[:, :, :] = np.array(final_probability_of_green)
                vehicle_distribution[:, :, :] = np.array(final_vehicle_distribution)

                # Offset Adjustment
                make_adjustments(green_aog, vehicle_distribution, offsetAdj)

                return -arrival_profile_optimize(green_aog, vehicle_distribution)

        if(method_to_use==2):
        # Reset matrix
                capacity[:, :, :] = np.array(fin_capacity)
                vehicle_distribution[:, :, :] = np.array(final_vehicle_distribution)

                # Offset Adjustment
                make_adjustments(capacity, vehicle_distribution, offsetAdj)

                return calculate_delay(capacity, vehicle_distribution)

        if(method_to_use==3):
                reset_green()
                green_adjustments(green, offsetAdj)

                return  -maximize_bandwidth(green)

        if(method_to_use==4):

                reset_green()
                green_adjustments(PrOG, offsetAdj)

                return  -maximize_bandwidth(PrOG)

        if(method_to_use==5):
        # Reset matrix
                capacity[:, :, :] = np.array(fin_capacity)
                vehicle_distribution[:, :, :] = np.array(final_vehicle_distribution)

                # Offset Adjustment
                make_adjustments(capacity, vehicle_distribution, offsetAdj)

                return number_of_stops(capacity, vehicle_distribution)

        if(method_to_use==6):
        # Reset matrix
                capacity[:, :, :] = np.array(fin_capacity)
                vehicle_distribution[:, :, :] = np.array(final_vehicle_distribution)

                # Offset Adjustment
                make_adjustments(capacity, vehicle_distribution, offsetAdj)

                return calculate_delay(capacity, vehicle_distribution) + 20/3600 * number_of_stops(capacity, vehicle_distribution)

        if(method_to_use==7):
        # Reset matrix
                green_AOG_10 = np.array(final_Prog_10)
                vehicle_distribution[:, :, :] = np.array(final_vehicle_distribution)

                # Offset Adjustment
                make_adjustments(green_AOG_10, vehicle_distribution, offsetAdj)

                return -arrival_profile_optimize(green_AOG_10, vehicle_distribution)

        if(method_to_use==8):
        # Reset matrix
                green_aog[:, :, :] = np.array(final_probability_of_green)
                vehicle_distribution[:, :, :] = np.array(final_vehicle_distribution)

                reset_green()
                green_adjustments(PrOG, offsetAdj)

                if (maximize_bandwidth(PrOG)<=30): return 0

                # Offset Adjustment
                make_adjustments(green_aog, vehicle_distribution, offsetAdj)

                return -arrival_profile_optimize(green_aog, vehicle_distribution)

    
 

I have developed this for a simple function where the obejective is to maximize green distribution overlaps. Vehicle arrivals were not considerd.
Later I have used hill climbing algorithm to maximize the green distribution. Initial adjustments are ranged $\begin{pmatrix}0 & 0 & \dots & 0 \end{pmatrix}$ for $1,2,\dots, n$ intersections. At each iteration step possible adjustments are $\begin{pmatrix} -40 & -15 & -5 & -1 & 1 & 5 & 15 & 40 \end{pmatrix}$. Goal of the Hill-climb function is $CycleLength - (b_1+b_2)$ where $b_i$ represents the width of green distribution. 

<h1>Optimization Algorithms</h1>

In [26]:
### Hill-Climbing Algorithm

# to get faster speed stop the plotting

def mutate_solution(solution): 
        index = np.random.randint(0,number_of_intersection)

        change = neighbors[np.random.randint(0,len(neighbors))]
        solution[index] =  np.mod(solution[index]+ change, cycle_length)


def hill_Climbing_algorithm(offsetAdj, method_to_use, neighbors,number_of_iteration, number_of_random_start, plotting):
    
        fig, axes = plt.subplots(1, 1)
        axes.set_xlabel("Iteration Number")
        axes.set_ylabel("Performance Index")

        axes.grid(which = 'both', linestyle = 'dashed')
        axes.grid(b=True, which='minor', alpha=0.2)
        axes.minorticks_on()
        #     axes.set_xlim([0, number_of_iteration])

        np.random.seed(1234)
        best = np.zeros((number_of_random_start, number_of_intersection), dtype = np.int32)
        best_score = np.zeros(number_of_random_start)

        ys = []
        xs = []

        i = 0
        j = 0
        rand_start = 0

        #     offsetAdj = np.zeros(number_of_intersection, dtype = np.int32)

        best[rand_start, :] = offsetAdj
        best_score[rand_start] = performance(offsetAdj, method_to_use)
    
        while(1):

                mutate_solution(offsetAdj)

                score = performance(offsetAdj, method_to_use)

                if(plotting): 
                        axes.scatter(i, score, color = 'black', s = 2)
                        if(rand_start%2): axes.scatter(i, best_score[rand_start], color = 'red', s = 2)
                        else: axes.scatter(i, best_score[rand_start], color = 'blue', s = 2)
                        fig.canvas.draw()
                #         print(best)

                if score < best_score[rand_start]:
                        best[rand_start, :] = offsetAdj
                        best_score[rand_start] = score

                        j = 0

                        xs.append(i)
                        ys.append(best_score[rand_start])


                offsetAdj[:] = best[rand_start, :] 
                #         print(i, " ", j, " ", offsetAdj[:])
                i += 1
                j += 1

                if(j>=number_of_iteration):
                        rand_start += 1
                        j = 0
                        if(rand_start>=number_of_random_start): break

                        offsetAdj[:] = np.random.randint(number_of_intersection)

                        best[rand_start, :] = offsetAdj
                        best_score[rand_start] = performance(offsetAdj, method_to_use)


        if(plotting==0): axes.plot(xs, ys)
        print(best_score)
        
        max_idx = np.argmin(best_score)
        
        offsetAdj[:] = best[max_idx, :]


<h1>Result and outputs</h1>

In [27]:
def print_output(offsetAdj): 
        reset_adjustments(green_aog, vehicle_distribution, capacity)
        veh_dist = np.array(final_vehicle_distribution)
        veh_dist_placeholder = np.array(final_vehicle_distribution)

        make_adjustments(green_aog, vehicle_distribution, offsetAdj)
        make_adjustments(capacity, veh_dist_placeholder, offsetAdj)

        sys_aog = arrival_profile_optimize(green_aog, vehicle_distribution)
        sys_delay = calculate_delay(capacity, vehicle_distribution)
        sys_Number_Stops = number_of_stops(capacity, vehicle_distribution)

        reset_green()
        green_adjustments(green, offsetAdj)
        sys_bandwidth = maximize_bandwidth(green)

        green_adjustments(PrOG, offsetAdj)
        sys_pog = maximize_bandwidth(PrOG)

        print("System AOG: %.2f" %sys_aog)
        print("System POG: %.2f" %sys_pog)
        print("System Delay: %.2f" %sys_delay)
        print("System Bandwidth: %.0f" %sys_bandwidth)
        print("System Number of Stops: %.2f" %sys_Number_Stops)
    

In [28]:
def offset_values_shift(offset_values, controller_start_number):
        offset_values = np.mod(offset_values - offset_values[0], cycle_length)
        for i in range(number_of_intersection):
                print(controller_start_number+i, " " , offset_values[i])

        return offset_values

In [68]:
offsets= np.zeros(number_of_intersection)
method_to_use = 1
neighbors = [-40,-15,-5,-1,1,5,15,40]
number_of_iteration = 1000
number_of_random_start = 10
plotting = 0

hill_Climbing_algorithm(offsets, method_to_use, neighbors,number_of_iteration,number_of_random_start, plotting)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[-13996.26666667 -13938.8        -13985.76666667 -13989.33333333
 -13987.66666667 -14004.5        -13921.9        -13928.2
 -13928.93333333 -13974.56666667]


In [69]:
print(offsets)
print_output(offsets)
offsets_2 = offset_values_shift(offsets, 9005)
print_output(offsets_2)

[ 2.  2.  2. 12.  1.  4. 17. 54.]
System AOG: 14004.50
System POG: 10.72
System Delay: 30.64
System Bandwidth: 19
System Number of Stops: 5217.13
9005   0.0
9006   0.0
9007   0.0
9008   10.0
9009   119.0
9010   2.0
9011   15.0
9012   52.0
System AOG: 14004.50
System POG: 10.72
System Delay: 30.64
System Bandwidth: 19
System Number of Stops: 5217.13


In [ ]:
def plot_offset_shift(offset_values, directory, figure_name, x_low, x_up):
        fig, axes = plt.subplots(1, 2,sharey = True, figsize=(12,6))

        offset_values = np.array(offset_values, dtype = np.int16)
        green_matrix = np.array(green_time[:, :, :simulation_time])

        fig.tight_layout()
        eb = 0

        for i in range(green_matrix.shape[0]):

                location = input_parameters["d1"][eb]

                placeHolder_greenMatrix = np.zeros(simulation_time)
        
                for k in range(simulation_time): 
                        placeHolder_greenMatrix[(k+offset_values[i])%simulation_time] = green_matrix[i][0][k]
        
                green_eb = np.array(placeHolder_greenMatrix)

                xs = np.multiply(np.arange(1, 7201), (1 - green_eb))
                ys= np.ones(7200)*location

                axes[0].scatter(xs, ys, s = 1, color = 'r')

                veh_speed_line_xs = [2000,2700]
                veh_speed_line_ys = [0,10955]

                axes[0].plot(veh_speed_line_xs, veh_speed_line_ys, color = 'g', linestyle = '--', linewidth = 0.7)

                eb += 2


        wb = 1
        for i in range(green_matrix.shape[0]):

                location = input_parameters["d1"][wb]

                placeHolder_greenMatrix = np.zeros(simulation_time)

                for k in range(simulation_time): 
                        placeHolder_greenMatrix[(k+offset_values[i])%simulation_time] = green_matrix[i][1][k]


                green_wb = np.array(placeHolder_greenMatrix)

                xs = np.multiply(np.arange(1, 7201), (1 - green_wb))
                ys= np.ones(7200)*location

                axes[1].scatter(xs, ys, s = 1, color = 'r')

                veh_speed_line_xs = [2200,2700]
                veh_speed_line_ys = [3500,3500-500*15.65]

                axes[1].plot(veh_speed_line_xs, veh_speed_line_ys, color = 'g', linestyle = '--', linewidth = 0.7)

                wb += 2

        axes[0].set_xlim(x_low, x_up)
        axes[1].set_xlim(x_low, x_up)

        axes[0].set_ylim(0, 2000)
        axes[1].set_ylim(0, 2000)

        axes[0].set_title("Eastbound")
        axes[1].set_title("Westbound")


        fig.savefig(directory+figure_name+".png", dpi = 500,  bbox_inches='tight')

In [ ]:
direc_tory = "C:\\Users\\ashams\\Box\\research works\\comparison signal offset optimization\\final figure and tables\\"

offsets = np.zeros(number_of_intersection, np.int16)
offsets = np.array([  27,   19.,  17.,  27.,  11.,  1.,  13, 43.], np.int16)
plot_offset_shift(offsets, direc_tory, "aog_link_pivot", 1800,2700)

print_output(offsets)

In [ ]:
# directory  = "C:\\Users\\ashams\Box\\research works\\comparison signal offset optimization\\final figure and tables\\"
# fig, axes = plt.subplots(1, 2, figsize = (6.5, 3), gridspec_kw={'width_ratios': [2, 1]})
# fig.tight_layout()
# fig.subplots_adjust(wspace=0.3)

# xs_bin = np.arange(0,120)

# axes[0].imshow(vehicle_arrival_reshape[2][0], cmap = "binary")
# axes[1].bar(xs_bin, final_vehicle_distribution[2][0], color = 'black', width = 1, alpha = 0.8)

# axes[0].set_title("(a) Number of Vehicles", fontsize = 10, y = -0.53, pad = -14)
# axes[1].set_title("(b) Frequency of Vehicle Arrival", fontsize = 10, y = -0.2, pad = -14)

# axes[0].set_xlabel("Cycle second")
# axes[0].set_ylabel("Cycle number")

# axes[1].set_xlabel("Cycle second")
# axes[1].set_ylabel("Frequency")

# fig.savefig(directory + "vehicle arrival.png", dpi = 300, bbox_inches = "tight")


In [ ]:
# a = np.array([0,1,2,3, 4])

# fig, axes = plt.subplots(1, 1, figsize = (2.5, 1))
# axes.imshow(a.reshape(1, 5), cmap= "binary")
# plt.tick_params(left=False,
#                 labelleft=False)
# axes.set_xticks(a)
# axes.set_xticklabels(('0', '1', '2', '3', '4'), fontsize = 12)

In [ ]:
# import matplotlib as mpl
# cmap = mpl.colors.ListedColormap(['white', 'green'])
# bounds = [0., 0.5, 1.]
# norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

In [ ]:
# directory  = "C:\\Users\\ashams\Box\\research works\\comparison signal offset optimization\\final figure and tables\\"
# fig, axes = plt.subplots(1, 2, figsize = (6.5, 2), gridspec_kw={'width_ratios': [2, 1]})
# fig.tight_layout()
# fig.subplots_adjust(wspace=0.3)

# xs_bin = np.arange(0,120)

# axes[0].imshow(green_distribution[2][0], interpolation='none', cmap=cmap, norm=norm)
# axes[1].bar(xs_bin, final_probability_of_green[2][0], color = 'green', width = 1)

# axes[0].set_title("(a) Signal state matrix", fontsize = 10, y = -0.2, pad = -14)
# axes[1].set_title("(b) Probability of green", fontsize = 10, y = -0.2, pad = -14)

# axes[0].set_xlabel("Cycle second")
# axes[0].set_ylabel("Cycle number")

# axes[1].set_xlabel("Cycle second")
# axes[1].set_ylabel("Probability")

# fig.savefig(directory + "signal state.png", dpi = 300, bbox_inches = "tight")

In [ ]:
# fig, axes = plt.subplots(1,1, figsize = (3,3.5))

# axes.bar(xs_bin, final_probability_of_green[2][0], color = 'green', width = 1, alpha = 0.8)

# axes2 = axes.twinx()
# axes2.bar(xs_bin, final_vehicle_distribution[2][0], color = 'black', width = 1, alpha = 0.8)

# axes.set_ylabel("Probability of green")
# axes2.set_ylabel("Frequency")

# axes.set_xlabel("Cycle second")

# fig.savefig(directory + "combined.png", dpi = 300, bbox_inches = "tight")


In [126]:
# reset_adjustments(green_aog, vehicle_distribution, capacity)
# plot_distributions(green_aog, vehicle_distribution)

In [124]:
# green_aog2 = np.array(final_probability_of_green)
# vehicle_distribution2 = np.array(final_vehicle_distribution)

# make_adjustments(green_aog2, vehicle_distribution2, offsets)
# plot_distributions(green_aog2, vehicle_distribution2)
# print_output(offsets)

In [125]:
# green_aog = np.array(final_probability_of_green)
# vehicle_distribution = np.array(final_vehicle_distribution)

# make_adjustments(green_aog, vehicle_distribution, offsets_2)
# plot_distributions(green_aog, vehicle_distribution)
# print_output(offsets_2)

In [129]:
offsets = np.array([22, 22, 21, 93, 82, 77, 75, 27], np.int16)
print_output(offsets)
offsets_2 = offset_values_shift(offsets, 9005)
print_output(offsets_2)

System AOG: 14003.80
System POG: 18.99
System Delay: 40.69
System Bandwidth: 25
System Number of Stops: 7067.38
9005   0
9006   0
9007   119
9008   71
9009   60
9010   55
9011   53
9012   5
System AOG: 14003.80
System POG: 18.99
System Delay: 40.69
System Bandwidth: 25
System Number of Stops: 7067.38


In [64]:
direc_tory = "C:\\Users\\ashams\\Box\\research works\\comparison signal offset optimization\\final figure and tables\\"

fig, axes = plt.subplots(1, 3,  figsize = (6.5, 2.75))
fig.tight_layout()

n_bins = np.arange(0, 120, 5)

split = split_matrix[6, 1, :split_cycle_num[6][1]]
eog = eog_matrix[6, 1, 1:eog_cycle_num[6][1]]

bog = (eog- split)%120

axes[0].hist(eog, bins = n_bins, color= 'black', alpha = 0.7)
axes[1].hist(split, bins = n_bins, color= 'black', alpha = 0.7)
axes[2].hist(bog, bins = n_bins, color= 'black', alpha = 0.7)


axes[0].set_xlabel("Cycle Second")
axes[1].set_xlabel("Green Durations")
axes[2].set_xlabel("Cycle Second")

axes[0].set_ylabel("Frequency")

axes[0].set_title("End of green", fontsize = 10)
axes[1].set_title("Split Time", fontsize = 10)
axes[2].set_title("Start of green", fontsize = 10)

fig.savefig(direc_tory + "expected_values.png", dpi = 300, bbox_inches = "tight")
        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [280]:
def plot_distribution_method(probability_of_green, vehicle_distribution, ax_num):
        signal_seq = np.arange(0,cycle_length, 1)
        veh_seq = np.arange(0,cycle_length, 1)

        for i in range( 1,8):
                ax = axes[i-1][ax_num]
                
                ax.bar(signal_seq,probability_of_green[i][0], width = 1, color = 'green', alpha = 0.8)

                ax2=ax.twinx()
                ax2.bar(veh_seq,vehicle_distribution[i][0], width = 1, color = 'black', alpha = 0.6)
                if(i==3 or i==5): ax2.set_yticks([0, 10, 20, 30, 40, 50])
                else: ax2.set_yticks([0, 10, 20, 30, 40])
                
                if(ax_num!=3): ax2.set_yticklabels([])

In [328]:
fig, axes = plt.subplots(7, 4, sharex = True, sharey = True, figsize = (6.5, 8.5))
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [364]:
plot_distribution_method(green_aog, vehicle_distribution, 3)

In [365]:
fig.subplots_adjust(wspace=0.1, hspace=0.1)

In [366]:
titles = ["Zero-offset", "AOG", "PrOG", "MAXBAND"]

for i in range(4): 
        axes[0][i].set_title(titles[i], fontsize = 10)
        
for i in range(7): 
        axes[i][0].set_ylabel("Int. " + str(i+2))
        
fig.text(0.45, 0, "Cycle Second", fontsize = 10)
fig.text(-0.03, 0.45, "Green probability", rotation = 90, fontsize = 10)
fig.text(1.03, 0.45, "Vehicle frequency", rotation = 90, fontsize = 10)

directory = "C:\\Users\\ashams\\Box\\research works\\comparison signal offset optimization\\analysis\\"
fig.savefig(directory + "distribution.png", dpi = 300, bbox_inches ="tight")

In [72]:
# offsets = np.array([0,113,72,23,74,16,57,100], np.int16)
print_output(offsets)

System AOG: 15172.17
System POG: 23.31
System Delay: 30.64
System Bandwidth: 19
System Number of Stops: 5217.13


In [71]:
arrival_on_green(green, bin_VAdj)

array([ 2.,  2.,  2., 12.,  1.,  4., 17., 54.])